In [69]:
import pandas as pd
WORKSPACE = '/Users/houtonglei/OneDrive - stu.hqu.edu.cn/数学建模/2021美赛/'
DATASET_DESCRIPTION = WORKSPACE + '2021_ICM_ProblemC/2021MCM_ProblemC_DataSet.xlsx'

dataset_description = pd.read_excel(DATASET_DESCRIPTION)
dataset_description['Detection Date'] = dataset_description['Detection Date'].apply(lambda x:x.value/(10**9*3600*24))

In [70]:
import sklearn
import numpy as np
dataset_description = dataset_description.replace('Positive ID', 1)
dataset_description = dataset_description.replace('Negative ID', 0)
train_set = dataset_description[dataset_description['Lab Status'].isin([0, 1])]
train_dataset = sklearn.utils.Bunch()
train_dataset.data = train_set[['Detection Date', 'Latitude', 'Longitude']]
train_dataset.target = train_set['Lab Status']
train_dataset.target_names = ['neg', 'pos']
train_dataset.filenames = train_set['GlobalID']

In [71]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
x_train = std.fit_transform(train_dataset.data)
y_train = train_dataset.target.astype(np.int8)

In [72]:
"""from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(train_dataset.data, train_dataset.target, test_size=0.25)
x_train = std.fit_transform(x_train)
y_train = y_train.astype(np.int8)
x_val = std.transform(x_val)
y_val = y_val.astype(np.int8)"""

'from sklearn.model_selection import train_test_split\nx_train, x_val, y_train, y_val = train_test_split(train_dataset.data, train_dataset.target, test_size=0.25)\nx_train = std.fit_transform(x_train)\ny_train = y_train.astype(np.int8)\nx_val = std.transform(x_val)\ny_val = y_val.astype(np.int8)'

In [73]:
from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV()
clf.fit(x_train, y_train)

LogisticRegressionCV()

In [74]:
"""print(clf.score(x_val, y_val))
print(clf.coef_)"""

'print(clf.score(x_val, y_val))\nprint(clf.coef_)'

In [ ]:
train_proba = clf.predict_proba(train_dataset.data)
pd.DataFrame(train_proba,index=train_dataset.filenames.tolist(),columns=train_dataset.target_names).to_csv('result/t&l_embedding.csv', float_format='%.16f')

In [75]:
test_set = dataset_description[dataset_description['Lab Status'].isin(['Unverified','Unprocessed'])]
test_dataset = sklearn.utils.Bunch()
test_dataset.data = test_set[['Detection Date', 'Latitude', 'Longitude']]
test_dataset.target = test_set['Lab Status']
test_dataset.target_names = ['Unknown']
test_dataset.filenames = test_set['GlobalID']

test_dataset.data = std.transform(test_dataset.data)

In [76]:
pred_proba = clf.predict_proba(test_dataset.data)
pd.DataFrame(pred_proba,index=test_dataset.filenames.tolist(),columns=train_dataset.target_names).to_csv('result/t&l_result.csv', float_format='%.16f')
